In [70]:
from pyspark.sql import SQLContext
import csv
from collections import defaultdict
import glob

cols = None
allR = []
adCarrier = {}
adDest = {}
adOrig = {}
adComb = {}

fList = glob.glob("../src/full/*.csv")
activeAirlines_l = [l.strip() for l in open("cur_airlines.txt","r").readlines()]
activeAirlines = dict( zip(activeAirlines_l, range(len(activeAirlines_l))) )

#         print '->', row[cols['ARR_DELAY']]        
#             if row[cols['']] != '':
#                 print row[cols['DIV_AIRPORT_LANDINGS']] 

def addDelay(curDelay, dStore, key):
    if key not in dStore:
        dStore[key] = (0., 0)
    curT, curN = dStore[key]
    if curDelay == '': # Ignore this flight
        return
    
    p_adelay = max(0., float(curDelay)) # Ignore early arrivals
    dStore[key] = (curT + p_adelay, curN + 1)

for fName in fList:
    print 'Processing', fName
    isFirst = True
    with open(fName, 'rb') as csvfile:
        line = csv.reader(csvfile, delimiter=',')
        for row in line:
            if cols is None:
                cols = dict( zip(row, range(len(row))) )
            
            if isFirst: # Header row
                isFirst = False
                continue
            
            curCarrier = row[cols['UNIQUE_CARRIER']]
            curDAirport = row[cols['DEST']]
            curOAirport = row[cols['ORIGIN']]
            curArrDelay = row[cols['ARR_DELAY']].strip()    
            currDepDelay = row[cols['DEP_DELAY']].strip()
            
            if curCarrier not in activeAirlines: # Only include active airlines
                continue
            
            addDelay(currDepDelay, adOrig, curOAirport)
            addDelay(curArrDelay, adDest, curDAirport)
            addDelay(curArrDelay, adCarrier, curCarrier)
            addDelay(currDepDelay, adComb, (curCarrier, curOAirport))


print cols
# print adComb
print len(adComb)
    

Processing ../src/full/328190688_T_ONTIME.csv
Processing ../src/full/328190842_T_ONTIME.csv
Processing ../src/full/328190997_T_ONTIME.csv
Processing ../src/full/328191155_T_ONTIME.csv
Processing ../src/full/328191317_T_ONTIME.csv
Processing ../src/full/328191485_T_ONTIME.csv
Processing ../src/full/328191637_T_ONTIME.csv
Processing ../src/full/328191796_T_ONTIME.csv
Processing ../src/full/328191964_T_ONTIME.csv
Processing ../src/full/328192142_T_ONTIME.csv
Processing ../src/full/328192276_T_ONTIME.csv
Processing ../src/full/328192412_T_ONTIME.csv
Processing ../src/full/328192596_T_ONTIME.csv
Processing ../src/full/328192763_T_ONTIME.csv
Processing ../src/full/328192917_T_ONTIME.csv
Processing ../src/full/328193095_T_ONTIME.csv
Processing ../src/full/328193280_T_ONTIME.csv
Processing ../src/full/328193432_T_ONTIME.csv
Processing ../src/full/328193574_T_ONTIME.csv
Processing ../src/full/328193754_T_ONTIME.csv
Processing ../src/full/328193907_T_ONTIME.csv
Processing ../src/full/328194093_T

In [71]:
def getTop(dDelay, count, worst = True, minN = 1):
    sArr = []
    for k in dDelay:
        if dDelay[k][1] < minN: # Below threshold
            continue
        sArr.append( (dDelay[k][0] / float(dDelay[k][1]), k) )
    sArr.sort(reverse = worst)
    
    return sArr[:count]

threshold = 3960 # 1 flight per day

worstDAirports = getTop(adDest, 10, True, threshold)
bestDAirports = getTop(adDest, 10, False, threshold)
print 'Worst destination airports:', worstDAirports
print 'Best destination airports:', bestDAirports

worstOAirports = getTop(adOrig, 10, True, threshold)
bestOAirports = getTop(adOrig, 10, False, threshold)
print 'Worst origin airports:', worstOAirports
print 'Best origin airports:', bestOAirports

worstCarriers = getTop(adCarrier, 10, True, threshold)
bestCarriers = getTop(adCarrier, 10, False, threshold)
print 'Worst carriers:', worstCarriers
print 'Best carriers:', bestCarriers

worstComb = getTop(adComb, 10, True, threshold)
bestComb = getTop(adComb, 10, False, threshold)
print 'Worst combiation:', worstComb
print 'Best combination:', bestComb

# print adDest['JFK']
# print adDest[worstDAirports[0][1]]
# adOrig
# adComb


Worst destination airports: [(19.33404255319149, 'SIT'), (19.174094301967997, 'ORD'), (18.39265536723164, 'GUC'), (17.835462058602555, 'GRR'), (17.587962962962962, 'JAC'), (17.508761329305138, 'DSM'), (17.50417702952618, 'EWR'), (17.42923794712286, 'CDV'), (17.365805818895065, 'EGE'), (17.22328548644338, 'YAK')]
Best destination airports: [(2.1744950681070927, 'ITO'), (3.235294117647059, 'PHF'), (4.867034990791897, 'LIH'), (5.471363468407439, 'KOA'), (5.844097995545657, 'EYW'), (6.468145735802972, 'OGG'), (6.787346221441124, 'MYR'), (8.29008424057541, 'HNL'), (8.869158878504672, 'ILM'), (8.947551909766727, 'CVG')]
Worst origin airports: [(24.48314606741573, 'GUC'), (23.652248150256117, 'JAC'), (20.392137096774192, 'SIT'), (19.517185994217797, 'EGE'), (19.287543869579984, 'ORD'), (19.234894991922456, 'JNU'), (17.46474358974359, 'PSG'), (16.896565656565656, 'HPN'), (15.91451731761238, 'HDN'), (15.64291072768192, 'GRR')]
Best origin airports: [(2.2491780178487555, 'ITO'), (3.8084358523725

In [69]:
# Active airlines
print activeAirlines

['M3', 'JP', 'A3', 'EI', 'NG', 'P5', 'SU', 'AR', '2K', 'AM', '8U', 'ZI', 'AH', 'G9', 'KC', 'UU', 'BT', 'AB', 'BP', 'SM', 'TY', 'AC', 'CA', 'XK', 'UX', 'AF', 'AI', 'JS', 'NX', 'MD', 'KM', 'MK', '9U', 'SW', 'NZ', 'PX', 'YW', 'JU', 'HM', 'VT', 'TN', 'TS', 'NF', 'RU', 'SB', '4Z', 'AS', 'AZ', 'NH', 'UJ', 'K4', 'AA', 'W3', 'IZ', 'OZ', '5Y', 'KK', 'AU', 'OS', 'AV', 'O6', 'J2', 'AD', 'JA', 'UP', 'PG', 'B2', '8H', 'BG', 'NT', 'BV', 'KF', 'BM', 'OB', 'BA', 'SN', 'FB', '5C', 'W8', 'CV', 'BW', 'V3', 'CX', 'CI', 'CK', 'MU', 'CF', 'CZ', 'WX', 'MN', 'DE', 'CM', 'XC', 'SS', 'OU', 'CU', 'OK', 'DL', 'D0', 'ES', 'Z6', 'D9', 'KA', 'MS', 'LY', 'EK', 'OV', 'ET', 'EY', 'YU', 'QY', 'EW', 'BR', 'FX', 'FJ', 'AY', 'BE', 'FH', 'GA', 'A9', 'ST', 'GF', 'HR', 'HU', 'HA', '5K', 'HX', 'UO', 'IB', 'FI', '7i', 'D6', '4O', '3L', 'IR', 'EP', '6H', 'JL', 'J9', '9W', 'S2', 'B6', 'R5', '5N', 'HO', 'KQ', 'Y9', 'KL', 'KE', 'KU', 'LR', 'TM', 'LA', '4M', 'UC', '4C', 'LP', 'XL', 'LI', 'N4', 'LO', 'LH', 'LH', 'CL', 'LG', 'W5', 'MH